<a href="https://colab.research.google.com/github/Nielspace/BERT/blob/master/Bert%3D%3EMultitext_Classification_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

     |████████████████████████████████| 675kB 8.3MB/s 
     |████████████████████████████████| 1.1MB 25.8MB/s 
     |████████████████████████████████| 890kB 31.1MB/s 
     |████████████████████████████████| 3.8MB 56.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=08b6f05498c87796fc336ae7250c57178f6176c6a511939d4085698f66655bb6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import transformers

class config:
    MODEL = 'bert-base-uncased'

    HIDDEN = 768

    MAX_LENGTH = 64
    TRAIN_BATCH_SIZE = 32
    VALID_BATCH_SIZE = 32

    EPOCHS = 2

    LR = (2e-5, 3e-5, 5e-5)
    EPS = 1e-8

    SEED = 23



In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import transformers
import torch
from torch.utils.data import Dataset, DataLoader



data = pd.read_json("News_Category_Dataset_v2.json", lines=True)

print(data)

data = data.dropna()
data = data.sample(n=20000)

data['text'] = data.headline + data.short_description
encoder = LabelEncoder()
data["classes"] = encoder.fit_transform((data['category']))
data = data[['text', 'category', 'classes']]


class TextClassificationDataset:

    def __init__(self,
                 texts,
                 labels):

        self.texts = texts
        self.labels = labels
        self.max_seq_length = config.MAX_LENGTH

        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            config.MODEL
        )

        self.sep_vid = self.tokenizer.vocab["[SEP]"]
        self.cls_vid = self.tokenizer.vocab["[CLS]"]
        self.pad_vid = self.tokenizer.vocab["[PAD]"]

    def __len__(self):

        return len(self.texts)

    def __getitem__(self, item):

        texts = str(self.texts[item])
        texts = " ".join(texts.split())
        inputs = self.tokenizer.encode_plus(
            texts,
            None,
            add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask=True,
            max_length=self.max_seq_length,
            return_token_type_ids=False,
            return_tensors="pt",
        )

        true_seq_length = len(inputs['input_ids'][0])
        pad_size = self.max_seq_length - true_seq_length
        pad_ids = torch.Tensor([self.pad_vid] * pad_size).long()
        ids = torch.cat((inputs['input_ids'][0], pad_ids))


        output_dict = {
            "ids": ids.flatten(),
            'attention_mask': inputs["attention_mask"][0].flatten(),
            'target' : torch.tensor(self.labels[item], dtype=torch.long)

        }

        

        return output_dict


SEED = 42

n_classes = len(data['classes'].unique())

train, val = train_test_split(
    data, test_size=0.30, random_state=SEED)


train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)




             category  ...       date
0               CRIME  ... 2018-05-26
1       ENTERTAINMENT  ... 2018-05-26
2       ENTERTAINMENT  ... 2018-05-26
3       ENTERTAINMENT  ... 2018-05-26
4       ENTERTAINMENT  ... 2018-05-26
...               ...  ...        ...
200848           TECH  ... 2012-01-28
200849         SPORTS  ... 2012-01-28
200850         SPORTS  ... 2012-01-28
200851         SPORTS  ... 2012-01-28
200852         SPORTS  ... 2012-01-28

[200853 rows x 6 columns]


In [10]:
from transformers import BertForSequenceClassification, BertModel
import torch
import torch.nn as nn





class Model(nn.Module):
    def __init__(self, n_classes):
        super(Model, self).__init__()
        self.bert = BertModel.from_pretrained(config.MODEL)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(config.HIDDEN, n_classes)

            
    
    def forward(self, input_ids, attention_mask=None):
        _, pooled_output = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        output = self.drop(pooled_output)
        # output = self.softmax(output)
        return self.out(output)

In [23]:
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader
import numpy as np
import time
import datetime
import random
from tqdm import tqdm


traindata = TextClassificationDataset(
    train.text.to_numpy(), 
    train.classes.to_numpy()
    )

trainLoader = DataLoader(
    traindata, 
    shuffle=True, 
    batch_size=config.TRAIN_BATCH_SIZE)


valdata = TextClassificationDataset(
    val.text.to_numpy(), 
    train.classes.to_numpy()
    )

valLoader = DataLoader(
    valdata, 
    shuffle=True, 
    batch_size=config.TRAIN_BATCH_SIZE)



if torch.cuda.is_available():
    device = torch.device("cuda")

    print("Cuda available")

else:
    print("No GPU's available")
    device = torch.device("cpu")

TOTAL_STEPS = len(trainLoader)*config.EPOCHS

model = Model(n_classes)
model.to(device)

optimizer = AdamW(
    model.parameters(),
    lr = config.LR[-1],
    eps=config.EPS
    )
scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=TOTAL_STEPS
)

def accuracy_check(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    label_flat = labels.flatten()
    return np.sum(pred_flat == label_flat) / len(label_flat)


def format_time(elasped):
    elasped_rounded = int(round(elasped))

    return str(datetime.timedelta(seconds=elasped_rounded))


random.seed(config.SEED)
np.random.seed(config.SEED)
torch.manual_seed(config.SEED)
torch.cuda.manual_seed(config.SEED)

training_stats = []

total_t0 = time.time()



def loss_fn(outputs, targets):
    return nn.CrossEntropyLoss()(outputs, targets.view(-1, 1))

for epoch in range(0, config.EPOCHS):
    print("")
    print("============EPOCHS {:}/{:}=============".format(epoch + 1, config.EPOCHS))
    print("Training")

    t0 = time.time()

    total_train_loss = 0

    model.train()
    for step, batch in enumerate(trainLoader):
       

        if step % 40 == 0 and not step == 0: 
            elapsed = format_time(time.time() - t0)


            print(" Batch {:>5,} of {:>5,}. Elasped: {:}".format(step, len(trainLoader), elapsed))

        b_input_ids = batch['ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        b_labels = batch['target'].to(device)
        
        model.zero_grad()

        output = model(
            b_input_ids,
            attention_mask = b_input_mask,
            )

        # output = torch.nn.functional.softmax(output, dim=1)

        # print(output)

        # lossfn = torch.nn.CrossEntropyLoss().to(device)
        loss = lossfn(output, b_labels).to(device)
        
        total_train_loss+=loss.item()



        loss.backward()


        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()


        avg_train_loss = total_train_loss/len(trainLoader)


        training_time = format_time(time.time() - t0)
        

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))


    #=========================================
    #           Validation
    #=========================================

    print("")
    print("Running Validation")
    
    t0 = time.time()

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_step = 0

    for batch in valLoader:

        b_input_ids = batch['ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        b_labels = batch['target'].to(device)

        with torch.no_grad():
            model.zero_grad()

            output = model(
                b_input_ids,
                attention_mask = b_input_mask,
                )

        # output = torch.nn.functional.softmax(output)

        # lossfn = torch.nn.CrossEntropyLoss().to(device)
        loss = lossfn(output, b_labels).to(device)
        
        total_eval_loss+=loss.item()

        output = output.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()


        total_eval_accuracy += accuracy_check(output, label_ids)

        
    
    avg_val_accuracy = total_eval_accuracy/len(valLoader)

    print("   Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss/len(valLoader)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            "epochs":epoch + 1,
            "Trainning Loss":avg_train_loss,
            "Valid Loss": avg_val_loss,
            "Valid Acc": avg_val_accuracy,
            "Training Time": validation_time
        }
    )


print("")
print("Training Complete")
print("Total Time took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
        

            









Cuda available

============EPOCHS 1/2=============
Training
 Batch    40 of   438. Elasped: 0:00:14
 Batch    80 of   438. Elasped: 0:00:28
 Batch   120 of   438. Elasped: 0:00:43
 Batch   160 of   438. Elasped: 0:00:57
 Batch   200 of   438. Elasped: 0:01:11
 Batch   240 of   438. Elasped: 0:01:26
 Batch   280 of   438. Elasped: 0:01:41
 Batch   320 of   438. Elasped: 0:01:55
 Batch   360 of   438. Elasped: 0:02:10
 Batch   400 of   438. Elasped: 0:02:25

  Average training loss: 2.15
  Training epoch took: 0:02:40

Running Validation
   Accuracy: 0.07
  Validation Loss: 4.79
  Validation took: 0:00:25

============EPOCHS 2/2=============
Training
 Batch    40 of   438. Elasped: 0:00:15
 Batch    80 of   438. Elasped: 0:00:31
 Batch   120 of   438. Elasped: 0:00:46
 Batch   160 of   438. Elasped: 0:01:02
 Batch   200 of   438. Elasped: 0:01:17
 Batch   240 of   438. Elasped: 0:01:33
 Batch   280 of   438. Elasped: 0:01:49
 Batch   320 of   438. Elasped: 0:02:04
 Batch   360 of   438.

In [21]:
output.shape

torch.Size([32, 41])